In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16,9]
plt.style.use('ggplot')
import os
import pandas as pd
import numpy as np 
import pickle as pk
import glob
from fbprophet import Prophet
import sys
sys.path.append('../')
import wiki
from wiki import utils 
import multiprocessing as mp
total_proc = None
from tqdm import tqdm
from IPython.display import clear_output

In [37]:
PROPHET_PATH = '../data/prophet/'
RESULTS_PATH = 'results/'
CACHE_PATH = 'cache/'
VERSION = 'v7/'

In [142]:
train = pd.read_csv('../data/'+'train_2.csv').drop('Page', axis=1);train.head()

,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,...,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31
0,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,24.0,...,37.0,58.0,25.0,23.0,25.0,47.0,44.0,26.0,24.0,21.0
1,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,4.0,...,14.0,34.0,16.0,31.0,34.0,33.0,32.0,29.0,34.0,25.0
2,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,4.0,...,6.0,23.0,11.0,8.0,6.0,13.0,12.0,8.0,12.0,9.0
3,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,16.0,...,21.0,42.0,23.0,17.0,11.0,21.0,20.0,20.0,110.0,15.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.0,41.0,25.0,22.0,20.0,24.0,29.0,20.0,17.0,24.0


In [140]:
train.to_feather('../data/train.f')

In [143]:
train2 = pd.read_feather('../data/train.f')

In [144]:
train.equals(train2)

True

In [148]:
def load_prophet_prop(VERSION, prop, force_remake=False, test=None):
    PROPHET_PATH = '../data/prophet/'
    CACHE_PATH = 'cache/'
    assert VERSION[-1] == '/'
    df_path = PROPHET_PATH+CACHE_PATH+VERSION[:-1]+prop+'.f'
    if os.path.isfile(df_path) and not force_remake:
        return pd.read_feather(df_path)
    else:
        df = pd.read_feather('../data/train.f')
        forecast_files = [x.split('/')[-1] for x in glob.glob(PROPHET_PATH+VERSION+'*df.f')]
        init_forc = pd.read_feather(PROPHET_PATH+VERSION+forecast_files[0])
        ds_min = init_forc.ds.min().date()
        ds_max = init_forc.ds.max().date()
        df = df.loc[:,str(ds_min):str(ds_max)]
        df.loc[:] = np.nan
        assert df.shape[1]-1 == (ds_max-ds_min).days
        for file in tqdm(forecast_files[:test]):
            forecast = pd.read_feather(PROPHET_PATH+VERSION+file)
            df.loc[int(file[:-4])] = forecast[prop].values
        df.sort_index(inplace=True)
        df = df.apply(pd.to_numeric)
        df.to_feather(df_path)
    return df

In [155]:
yhat = load_prophet_prop('v7t/', 'yhat')

In [156]:
yhat.shape

(145063, 793)

In [157]:
yhat

,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,...,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31
0,20.0,16.0,18.0,17.0,16.0,13.0,15.0,14.0,11.0,13.0,...,23.0,23.0,20.0,23.0,23.0,23.0,21.0,24.0,24.0,21.0
1,24.0,24.0,28.0,34.0,28.0,28.0,35.0,31.0,29.0,32.0,...,61.0,57.0,54.0,56.0,60.0,53.0,51.0,57.0,52.0,49.0
2,4.0,1.0,1.0,2.0,2.0,1.0,1.0,3.0,1.0,0.0,...,0.0,2.0,0.0,0.0,2.0,2.0,1.0,1.0,3.0,1.0
3,26.0,24.0,25.0,25.0,22.0,25.0,23.0,23.0,21.0,22.0,...,12.0,12.0,10.0,11.0,12.0,9.0,12.0,10.0,10.0,8.0
4,4.0,5.0,3.0,5.0,6.0,2.0,0.0,2.0,3.0,1.0,...,0.0,4.0,5.0,4.0,7.0,8.0,6.0,3.0,8.0,10.0
5,4.0,6.0,6.0,6.0,10.0,7.0,4.0,4.0,6.0,5.0,...,13.0,13.0,16.0,15.0,16.0,21.0,18.0,14.0,15.0,17.0
6,44.0,42.0,41.0,49.0,49.0,45.0,41.0,42.0,40.0,39.0,...,2.0,3.0,1.0,0.0,6.0,7.0,2.0,0.0,0.0,0.0
7,0.0,4.0,15.0,21.0,21.0,23.0,26.0,24.0,30.0,41.0,...,110.0,103.0,103.0,108.0,108.0,102.0,97.0,92.0,83.0,81.0
8,22.0,21.0,19.0,24.0,24.0,23.0,20.0,18.0,18.0,17.0,...,44.0,43.0,42.0,41.0,46.0,47.0,46.0,43.0,42.0,42.0
9,12.0,11.0,10.0,8.0,7.0,12.0,11.0,13.0,13.0,11.0,...,21.0,22.0,22.0,20.0,18.0,17.0,22.0,22.0,23.0,23.0


In [113]:
diff = train - yhat; diff.head()

,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,...,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31
0,-4.0,-8.0,-14.0,-6.0,-3.0,-5.0,-7.0,7.0,15.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-7.0,-5.0,-8.0,-11.0,-13.0,-11.0,-11.0,-18.0,-18.0,-27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2.0,-1.0,1.0,-1.0,-2.0,3.0,0.0,0.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.0,-13.0,-16.0,68.0,-19.0,1.0,-9.0,-13.0,-9.0,-5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
diff.mean(axis=1).head()

0    0.381992
1    0.645293
2    0.287858
3   -0.837653
4    2.038462
dtype: float64

In [136]:
diff.iloc[:,:-60].mean(axis=1).head()

0    0.381992
1    0.645293
2    0.287858
3   -0.837653
4    2.038462
dtype: float64

In [137]:
diff.iloc[:,:-61].mean(axis=1).head()

0    0.389344
1    0.659836
2    0.286885
3   -0.827869
4    2.006803
dtype: float64

In [133]:
diff.iloc[:,:-120].mean(axis=1).head()

0    0.011887
1   -0.002972
2   -0.034175
3    0.000000
4   -0.060209
dtype: float64

In [134]:
diff.iloc[:,:-300].mean(axis=1).head()

0    0.192698
1   -0.058824
2   -0.105477
3    0.070994
4   -0.024752
dtype: float64